In [1]:
import os
import pandas as pd
import numpy as np
from collections import defaultdict

In [2]:
path_before_clu = '../data/processed/01_fasta_seq'

path_fast_tree = '../data/processed/05_seq_for_tree/seq_for_tree_v7.fasta'

 Для каждой последовательности (ID), указанной в файле `path_fast_tree`, определить, в каких файлах из папки `../data/processed/01_fasta_seq` (с расширением .fasta или .faa) она содержится, и сохранить это соответствие в CSV-файл.

# declaring functions

In [3]:
def get_ids_from_file(file_path):
    """Извлекает ID последовательностей из FASTA файла."""
    ids = []
    with open(file_path, 'r') as f:
        for line in f:
            if line.startswith('>'):
                seq_id = line[1:].split()[0].strip()
                ids.append(seq_id)
    return ids


In [4]:
def parse_sources_for_ids(target_ids, path):
    """Ищет источники (файлы) для заданных ID в указанных папках."""
    # Словарь для хранения найденных источников
    id_sources = defaultdict(set)
    
    # Поиск в первой папке (path0)
    for filename in os.listdir(path):
        if filename.endswith('.faa') or filename.endswith('.fasta'):
            file_path = os.path.join(path, filename)
            file_ids = set(get_ids_from_file(file_path))
            # Пересечение ID файла с целевыми ID
            for id_ in file_ids & target_ids:
                id_sources[id_].add(filename)
    
    return id_sources


In [5]:
def custom_rename(source_str):
    if not isinstance(source_str, str) or not source_str:
        return source_str

    # Шаг 1: Преобразуем 'hit_table_' → '_homologues'
    parts = []
    for part in source_str.split(';'):
        if part.startswith('hit_table_') and part.endswith('_all.fasta'):
            clean_name = part.replace('hit_table_', '').replace('_all.fasta', '')
            parts.append(f'{clean_name}_homologues')
        else:
            parts.append(part.replace('.fasta', '').replace('.faa', ''))  # убираем расширения

    # Шаг 2: Дополнительные переименования
    new_parts = []
    for part in parts:
        if part in ['old_vibrio_homologues', 'old_homologues']:
            new_parts.append('Old_homologues')
        elif part in ['HEC-02', 'HEC-03', 'HEC-04']:
            new_parts.append('HECs')
        elif part in ['Ea59_Lambda_homologues', 'PtuA_Slividans_homologues']:
            new_parts.append('Ea59&PtuA_homologues')
        elif 'PtuA' in part:
            new_parts.append('Ea59&PtuA')
        elif 'JetC_ECOR67_from_1338_to_1445_homologues' in part:
            new_parts.append('JetC_ECOR67_homologues')
        elif 'ABC_ATPase_PARIS' in part:
            new_parts.append('PARIS')
        else:
            new_parts.append(part)

    # Удаляем дубликаты, сортируем и возвращаем
    return ';'.join(sorted(set(new_parts)))


# file processing

In [6]:
# Получаем целевые ID из файла path_fast_tree
target_ids = set(get_ids_from_file(path_fast_tree))


In [7]:
# Собираем источники для целевых ID
sources_dict = parse_sources_for_ids(target_ids, path_before_clu)


In [8]:
# Создаем DataFrame
df_data = []
for id_ in target_ids:
    sources = sources_dict.get(id_, set())
    df_data.append({
        'ID': id_,
        'sources': ';'.join(sorted(sources)) if sources else ''
    })

source_ID = pd.DataFrame(df_data)


In [9]:
source_ID

,ID,sources
0,WP_348782179.1,hit_table_NsnB_ECOR37_all.fasta
1,WMJ72022.1,hit_table_NsnB_ECOR37_all.fasta
2,WP_354037498.1,hit_table_NsnB_ECOR37_all.fasta
3,NZ_CP090673.1_1627,PtuA.faa
4,NZ_CP091285.1_2191,GajA.faa
...,...,...
8440,HEV3317244.1,hit_table_NsnB_ECOR37_all.fasta
8441,CP051234.1_1523,GajA.faa
8442,OY759782.1_2977,PtuA.faa
8443,OQW90533.1,hit_table_HEC-01A_ECOR46_all.fasta


In [10]:
sorted(source_ID['sources'].unique())

['',
 'AAA_ATPases.fasta',
 'ABC_ATPase_PARIS.fasta',
 'AbiL.faa',
 'GajA.faa',
 'GajA.faa;PtuA.faa',
 'HEC-02.faa',
 'HEC-03.faa',
 'HEC-04.faa',
 'JetC_ECOR67_from_1338_to_1445_all.fasta',
 'LmuB.faa',
 'PDC-M01A.faa',
 'PDC-M01A.faa;PtuA.faa',
 'Ppl.faa',
 'PrrC.faa',
 'PtuA.faa',
 'hit_table_Ea59_Lambda_all.fasta',
 'hit_table_Ea59_Lambda_all.fasta;hit_table_PtuA_Slividans_all.fasta',
 'hit_table_HEC-01A_ECOR46_all.fasta',
 'hit_table_IbfA_all.fasta',
 'hit_table_NsnB_ECOR37_all.fasta',
 'hit_table_PD-T4_4A_ECOR58_all.fasta',
 'hit_table_PtuA_Slividans_all.fasta',
 'hit_table_old_all.fasta',
 'hit_table_old_vibrio_all.fasta']

In [11]:
source_ID['sources'] = source_ID['sources'].apply(custom_rename)


In [12]:
sorted(source_ID['sources'].unique())

['',
 'AAA_ATPases',
 'AbiL',
 'Ea59&PtuA',
 'Ea59&PtuA;GajA',
 'Ea59&PtuA;PDC-M01A',
 'Ea59&PtuA_homologues',
 'GajA',
 'HEC-01A_ECOR46_homologues',
 'HECs',
 'IbfA_homologues',
 'JetC_ECOR67_from_1338_to_1445_all',
 'LmuB',
 'NsnB_ECOR37_homologues',
 'Old_homologues',
 'PARIS',
 'PD-T4_4A_ECOR58_homologues',
 'PDC-M01A',
 'Ppl',
 'PrrC']

In [13]:
# Проверяем NA/пустые значения
empty_sources = source_ID[source_ID['sources'].isin(['', None, np.nan])]
if not empty_sources.empty:
    print("Найдены ID с отсутствующими источниками:")
    print(empty_sources['ID'].tolist())
else:
    print("Пропущенных источников не обнаружено")


Найдены ID с отсутствующими источниками:
['HEC-01A_ECOR46', 'NsnB_ECOR37']


In [14]:
# Ручные обновления для ID с NA
source_ID.loc[source_ID['ID'] == 'NsnB_ECOR37', 'sources'] = 'NsnB_ECOR37_homologues'
source_ID.loc[source_ID['ID'] == 'HEC-01A_ECOR46', 'sources'] = 'HEC-01A_ECOR46_homologues'


In [15]:
# Проверяем NA/пустые значения
empty_sources = source_ID[source_ID['sources'].isin(['', None, np.nan])]
if not empty_sources.empty:
    print("Найдены ID с отсутствующими источниками:")
    print(empty_sources['ID'].tolist())
else:
    print("Пропущенных источников не обнаружено")


Пропущенных источников не обнаружено


In [16]:
# Сохраняем
source_ID.to_csv("../results/result_sources_ID.csv", index=False)
